In [1]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
from matplotlib.animation import FuncAnimation
from pylab import rcParams
rcParams['figure.figsize'] = 5,5
rcParams['xtick.labelsize'] = 10
rcParams['ytick.labelsize'] = 10
import os 
my_path = os.path.abspath('/Users/juan/Desktop/Git/3level/DATA/')



In [2]:
class variables():
    def __init__(self):
         #eigen energies
        self.w10 = 20
        self.w2w1 = 1.6
        self.w20 = self.w10*self.w2w1
        #carnot efficiency
        self.n_c  =np.arange(1e-6,1,0.02)
        #temperatures
        self.b_l  = 1/np.array([150,50,5])[2] 
        self.b_h  = (1-self.n_c)*self.b_l
       #energy variables
        self.lw = np.linspace(0,np.sqrt(self.w2w1),100)
        self.e_20 = self.w10*((1+self.w2w1 )/2 + np.sqrt(((self.w2w1-1)/2 )**2 +(self.lw)**2  ))
        self.e_10 = self.w10*((1+self.w2w1 )/2 - np.sqrt(((self.w2w1-1)/2 )**2 +(self.lw)**2  ))
       #lindblad constants
        self.g_l10  = 100
        self.g_h10 =  self.g_l10/2
        self.g_h20  = self.g_l10*5
        self.g_l20 = self.g_h20/2
       
        #cosine data
        self.c    = (self.w2w1 -1 )/(2*np.sqrt(((self.w2w1-1)/2 )**2 +(self.lw)**2 ))
        #power variables
    def g1(self):
        return 1/2*(self.g_l10*(1+self.c) + self.g_h10*(1-self.c))
    def g2(self):
        return 1/2*(self.g_l20*(1-self.c) + self.g_h20*(1+self.c)) 
    def g1m(self):
        result=[]
        for value in range(len(self.b_h)):
            result.append(1/2*(self.g_l10*(1+self.c)*np.exp(-self.b_l*self.e_10)\
                        + self.g_h10*(1-self.c)*np.exp(-self.b_h[value]*self.e_10)))
        return result
    def g2m(self):
        result=[]
        for value in range(len(self.b_h)):
            result.append(1/2*(self.g_l20*(1-self.c)*np.exp(-self.b_l*self.e_20) \
                        + self.g_h20*(1+self.c)*np.exp(-self.b_h[value]*self.e_20)))
        return result

    def omega(self):
        return ((self.e_20-self.e_10)**2 + ( (self.g1()+self.g2())/2)**2)\
                /(self.w20)

    def G(self):
        return  1/2*(self.g1()+self.g2())*(1 + (1/(self.e_20-self.e_10)**2)\
                *(1/2*(self.g1()+self.g2()))**2)

    def q1(self):
        return (var.g_h10*((1-var.c)/(1+var.c)) )/(var.g_l10+ var.g_h10*((1-var.c)/(1+var.c))) 
    
    def q2(self):
        return (var.g_l20*((1-var.c)/(1+var.c)) )/(var.g_h20+ var.g_l20*((1-var.c)/(1+var.c))) 
     
    def nq_max(self):
        result =[]
        for value in range(len(self.b_h)):
            result.append((1- var.e_10/var.e_20)*( (np.exp(-self.b_h[value]*self.e_20) \
                -np.exp(-self.b_l*self.e_20) )/(np.exp(-self.b_l*self.e_10) \
                -np.exp(-self.b_l*self.e_20))) )
        return result 


 #Initialize the object var       
var = variables()



var.n_c[49]




0.980001

In [4]:
       
 def POWER(var):
    e_20 = var.e_20
    e_10 = var.e_10
    c= var.c
    g2m = var.g2m()
    g1m = var.g1m()
    g2 = var.g2()
    g1 = var.g1()
    omega = var.omega()
    w21 = var.w20- var.w10
    result =[]
    for value in range(len(var.b_h)):
        result.append( (1/2*(e_20-e_10)*(1-c**2)*omega\
                      *( (g2m[value]/g2) - (g1m[value]/g1)))\
                /( (1/2+(g2m[value]/g2))\
               *( ((1-c**2)*omega)/g1\
               +(w21/((e_20-e_10)**2))*(1/2*(g1+g2)))\
               +(1/2+(g1m[value]/g1))\
               * ( ((1-c**2)*omega)/g2 \
               +(w21/((e_20-e_10)**2))*(1/2*(g1+g2)) )\
                ))
    return np.array(result)


def P0(var):
    e_20 = var.e_20
    e_10 = var.e_10
    c= var.c
    g_l10  =  var.g_l10
    g_l20  =  var.g_l20 
    g_h20  =  var.g_h20  
    g_h10  =  var.g_h10 
    G_l10  =  var.g_l10/(var.g_l10+ var.g_h10*((1-var.c)/(1+var.c))) 
    G_l20  =  (var.g_l20*((1-var.c)/(1+var.c)) )/(var.g_h20+ var.g_l20*((1-var.c)/(1+var.c))) 
    G_h10  =  (var.g_h10*((1-var.c)/(1+var.c)) )/(var.g_l10+ var.g_h10*((1-var.c)/(1+var.c))) 
    G_h20  =  var.g_h20/(var.g_h20+ var.g_l20*((1-var.c)/(1+var.c))) 
    b_h    = var.b_h
    b_l    = var.b_l
    result = []
    for value in range(len(var.b_h)):
        result.append( e_10*g_h10*G_l10*((1-c)/2)*(np.exp(-b_h[value]*e_10)-np.exp(-b_l*e_10))
                      +e_20*g_h20*G_l20*((1+c)/2)*(np.exp(-b_h[value]*e_20)-np.exp(-b_l*e_20) ))
    return result

def R00(var):
    e_20 = var.e_20
    e_10 = var.e_10
    c= var.c
    g2m = var.g2m()
    g1m = var.g1m()
    g2 = var.g2()
    g1 = var.g1()
    omega = var.omega()
    w21 = var.w20- var.w10
    result =[]
    for value in range(len(var.b_h)):
        result.append( ( ((1-c**2)*omega)/g1\
               +(w21/((e_20-e_10)**2))*(1/2*(g1+g2)) +\
                ((1-c**2)*omega)/g2 \
               +(w21/((e_20-e_10)**2))*(1/2*(g1+g2)) ) \
                /( (1/2+(g2m[value]/g2))\
               *( ((1-c**2)*omega)/g1\
               +(w21/((e_20-e_10)**2))*(1/2*(g1+g2)))\
               +(1/2+(g1m[value]/g1))\
               * ( ((1-c**2)*omega)/g2 \
               +(w21/((e_20-e_10)**2))*(1/2*(g1+g2)) )\
                ))
    return result



def Ql(var,p,qd):
    e_20 = var.e_20
    e_10 = var.e_10
    c= var.c
    g_l10  =  var.g_l10
    g_l20  =  var.g_l20 
    g_h20  =  var.g_h20  
    g_h10  =  var.g_h10 
    G_l10  =  var.g_l10/(var.g_l10+ var.g_h10*((1-var.c)/(1+var.c))) 
    G_l20  =  (var.g_l20*((1-var.c)/(1+var.c)) )/(var.g_h20+ var.g_l20*((1-var.c)/(1+var.c))) 
    G_h10  =  (var.g_h10*((1-var.c)/(1+var.c)) )/(var.g_l10+ var.g_h10*((1-var.c)/(1+var.c))) 
    G_h20  =  var.g_h20/(var.g_h20+ var.g_l20*((1-var.c)/(1+var.c))) 
    return ( (e_20/(e_20 -e_10))*G_l20 -(e_10/(e_20 -e_10))*G_l10 )*p -qd

def Qh(var,p,qd):
    e_20 = var.e_20
    e_10 = var.e_10
    c= var.c
    g_l10  =  var.g_l10
    g_l20  =  var.g_l20 
    g_h20  =  var.g_h20  
    g_h10  =  var.g_h10 
    G_l10  =  var.g_l10/(var.g_l10+ var.g_h10*((1-var.c)/(1+var.c))) 
    G_l20  =  (var.g_l20*((1-var.c)/(1+var.c)) )/(var.g_h20+ var.g_l20*((1-var.c)/(1+var.c))) 
    G_h10  =  (var.g_h10*((1-var.c)/(1+var.c)) )/(var.g_l10+ var.g_h10*((1-var.c)/(1+var.c))) 
    G_h20  =  var.g_h20/(var.g_h20+ var.g_l20*((1-var.c)/(1+var.c))) 
    return ( (e_20/(e_20 -e_10))*G_h20 -(e_10/(e_20 -e_10))*G_h10 )*p +qd

def EFF(p,q):
    return p/q


def n_q(var):
    e_20 = var.e_20
    e_10 = var.e_10
    q1 = var.q1()
    q2 = var.q2()
    return (1- e_10/e_20)/(1- q2 -(e_10/e_20)*q1 )

    
        


In [5]:
var.g_h10 =  var.g_l10*(0.5)
var.g_l20 = var.g_h20*(0.5)
x =var.lw/max(var.lw)
P   = POWER(var)
R   = np.array(R00(var))
P_0 = np.array(P0(var))
Q_d = R*P_0
Q_l = Ql(var,P,Q_d)
Q_h = Qh(var,P,Q_d)
n   = EFF(P,Q_h) 
n2  = EFF(P,Q_l) 

a =np.transpose(P)
b = np.transpose(Q_h)
c = np.transpose(Q_l)
d = np.transpose(Q_d)
e = np.transpose(n)

b[39]>c[39]
c[39]


<ipython-input-4-3c322c663100>:101: RuntimeWarning: invalid value encountered in true_divide
  return p/q


array([ 1.86591258e+01,  1.74776616e+01,  1.61901036e+01,  1.47843524e+01,
        1.32466654e+01,  1.15614148e+01,  9.71080895e+00,  7.67457253e+00,
        5.42957978e+00,  2.94943378e+00,  2.03985070e-01, -2.84121851e+00,
       -6.22555923e+00, -9.99406331e+00, -1.41982109e+01, -1.88968521e+01,
       -2.41572380e+01, -3.00561774e+01, -3.66813245e+01, -4.41326048e+01,
       -5.25237778e+01, -6.19841340e+01, -7.26603098e+01, -8.47181978e+01,
       -9.83449084e+01, -1.13750722e+02, -1.31170941e+02, -1.50867522e+02,
       -1.73130317e+02, -1.98277716e+02, -2.26656428e+02, -2.58640058e+02,
       -2.94626115e+02, -3.35031009e+02, -3.80282584e+02, -4.30809735e+02,
       -4.87028737e+02, -5.49326046e+02, -6.18037601e+02, -6.93424985e+02,
       -7.75649327e+02, -8.64744340e+02, -9.60590516e+02, -1.06289300e+03,
       -1.17116602e+03, -1.28472668e+03, -1.40270058e+03, -1.52404064e+03,
       -1.64755927e+03, -1.77197229e+03])

In [ ]:
%matplotlib nbagg
var = variables()
step = np.append(np.arange(0,0.08,0.001),np.arange(0.08,10,0.1))
step = np.append(step,np.arange(10,101,.50))
fig  = plt.figure()
#(ax1, ax2), ax3 = axs
ax1 = fig.add_subplot(2, 2, 1)
ax2 = fig.add_subplot(2, 2, 2)
ax3 = fig.add_subplot(2, 2, 3)
ax4 = fig.add_subplot(2, 2, 4)
plt.style.use('seaborn-pastel')

        
ax1.set_xlabel(r'$\Lambda/\Lambda_{max}$', fontsize =10)
ax1.set_ylabel(r'$\eta$',fontsize =10)
line1, = ax1.plot([],[],color='black',lw=2)
ax1.axhline(y=1-var.b_h[39]/var.b_l, color='r', linestyle='--')
ax1.set_xlim(0,1)
ax1.set_ylim(-0,1)
ax1.set_xticks([0,0.2,0.4,0.6,0.8,1])

    
        
ax2.set_xlabel(r'$\Lambda/\Lambda_{max}$',fontsize =10)
ax2.set_ylabel(r'$P$',fontsize =10)
line2, = ax2.plot([],[],color='blue',lw=2)
#ax1.set_aspect('equal', 'datalim')
ax2.set_xlim(0,1)
ax2.set_ylim(-0,480)
ax2.set_xticks(np.arange(0,1,0.2))
#ax2.set_title(r'$\eta_c=%.2f$'%(1-var.b_h[39]/var.b_l))
ax2.set_xticks([0,0.2,0.4,0.6,0.8,1])

ax3.set_xlabel(r'$\eta$',fontsize =10)
ax3.set_ylabel(r'$P$',fontsize =10)
line3, = ax3.plot([],[],color='red',lw=2)
#ax1.set_aspect('equal', 'datalim')
ax3.set_xlim(0,1)
ax3.set_ylim(-0,480)
ax3.set_xticks(np.arange(0,1,0.2))
#ax3.set_title(r'$\eta_c=%.2f$'%(1-var.b_h[39]/var.b_l))
ax3.set_xticks([0,0.2,0.4,0.6,0.8,1])
ax3.axvline(x=1-var.b_h[49]/var.b_l, color='r', linestyle='--')

ax4.set_xlabel(r'$\Lambda/\Lambda_{max}$',fontsize =10)
ax4.set_ylabel(r'$max\eta_{q}$',fontsize =10)
line4, = ax4.plot([],[],color='red',lw=2)
line5, = ax4.plot([],[],color='blue',lw=2)

#ax1.set_aspect('equal', 'datalim')
ax4.set_xlim(0,1)
ax4.set_ylim(0.95,1)
ax4.set_xticks(np.arange(0,1,0.2))
#ax3.set_title(r'$\eta_c=%.2f$'%(1-var.b_h[39]/var.b_l))
ax4.set_xticks([0,0.2,0.4,0.6,0.8,1])
ax4.axhline(y=1-var.b_h[49]/var.b_l, color='r', linestyle='--')
ax4.axhline(y=1, color='orange', linestyle='--')




def init():
    line1.set_data([],[])
    line2.set_data([],[])
    line3.set_data([],[])
    line4.set_data([],[])
    line5.set_data([],[])
    
    return line1,

def animate(i):
    x = var.lw/max(var.lw)
    
    var.g_h10 =  var.g_l10*(i/100)
    var.g_l20 = var.g_h20*(i/100)
    P   = POWER(var)
    R   = np.array(R00(var))
    P_0 = np.array(P0(var))
    Q_d = R*P_0
    Q_l = Ql(var,P,Q_d)
    Q_h = Qh(var,P,Q_d)
    n   = EFF(P,Q_h)
    y1   = n[49]   
    y2  = P[49]
    y3  = var.nq_max()[49]
    y4 = n_q(var)
    fig.suptitle(\
    r'$\frac{\gamma_h(\epsilon_{10})}{\gamma_h(\epsilon_{20})},\frac{\gamma_l(\epsilon_{20})}{\gamma_l(\epsilon_{10})}=%.4f$'%(i/100)\
               + r'$ \;\;\; \eta_c=%.2f$'%(1-var.b_h[49]/var.b_l), fontsize= 15)
   
    line1.set_data(x,y1)
    line1.set_label(r'$\eta$')
    line2.set_data(x,y2)
    line2.set_label(r'$\eta$')
    line3.set_data(y1,y2)
    line3.set_label(r'$P$')
    line4.set_data(x,y3)
    line4.set_label(r'$max\eta_q$')
    line5.set_data(x,y4)
    line5.set_label(r'$\eta_q$')
    ax1.legend(bbox_to_anchor=(1, .8), loc='lower right', borderaxespad=0, fontsize=10)
    ax2.legend(bbox_to_anchor=(1, .8), loc='lower right', borderaxespad=0, fontsize=10)
    ax3.legend(bbox_to_anchor=(1, .8), loc='lower right', borderaxespad=0, fontsize=10)
    ax4.legend(bbox_to_anchor=(1, .8), loc='lower right', borderaxespad=0, fontsize=10)
    
    plt.tight_layout(pad= 1,h_pad= .002,w_pad=.001)
   # plt.title(r'$\eta_c=%.2f$'%(1-var.b_h[39]/var.b_h))


    return line1,


anim = FuncAnimation(fig, animate, frames= step, interval=8, blit=True)
#anim.save('../../../3level_img/lambda_opt.gif', writer='imagemagick', fps=90)



In [ ]:

%matplotlib nbagg
var = variables()
step = np.append(np.arange(0,0.08,0.001),np.arange(0.08,10,0.1))
step = np.append(step,np.arange(10,101,.50))
#step = np.arange(0,1,0.1)
#fig, axs = plt.subplots(2, 3, gridspec_kw={'hspace': 1, 'wspace': 1})
fig  = plt.figure()
#(ax1, ax2), ax3 = axs
ax1 = fig.add_subplot(2, 2, 1)
ax2 = fig.add_subplot(2, 2, 2)
ax3 = fig.add_subplot(2, 2, 3)
ax4 = fig.add_subplot(2, 2, 4)
plt.style.use('seaborn-pastel')

 #Power  
ax1.set_xlabel(r'$\beta_h/\beta_l$', fontsize =10)
ax1.set_ylabel(r'$P$',fontsize =10)
line1, = ax1.plot([],[],color='black',lw=2)
ax1.axhline(y=1-var.b_h[39]/var.b_l, color='r', linestyle='--')
ax1.set_xlim(0,1)
ax1.set_ylim(-100,100)
ax1.set_xticks([0,0.2,0.4,0.6,0.8,1])

    
#Efficiency     
ax2.set_xlabel(r'$\beta_h/\beta_l$',fontsize =10)
ax2.set_ylabel(r'$\eta$',fontsize =10)
line2, = ax2.plot([],[],color='blue',lw=2)
line21, = ax2.plot([],[],color='red',lw=2)
#ax1.set_aspect('equal', 'datalim')
ax2.set_xlim(0,1)
ax2.set_ylim(-0,1)
ax2.set_xticks(np.arange(0,1,0.2))
#ax2.set_title(r'$\eta_c=%.2f$'%(1-var.b_h[39]/var.b_l))
ax2.set_xticks([0,0.2,0.4,0.6,0.8,1])

# Q_h
ax3.set_xlabel(r'$\beta_h/\beta_l$',fontsize =10)
ax3.set_ylabel(r'$Q_h$',fontsize =10)
line3, = ax3.plot([],[],color='red',lw=2)
line4, = ax3.plot([],[],color='green',lw=2)
#ax1.set_aspect('equal', 'datalim')
ax3.set_xlim(0,1)
ax3.set_ylim(-500,700)
ax3.set_xticks(np.arange(0,1,0.2))
#ax3.set_title(r'$\eta_c=%.2f$'%(1-var.b_h[39]/var.b_l))
ax3.set_xticks([0,0.2,0.4,0.6,0.8,1])
ax3.axhline(y=0, color='r', linestyle='--')


# Q_l
ax4.set_xlabel(r'$\beta_h/\beta_l$',fontsize =10)
ax4.set_ylabel(r'$Q_l$',fontsize =10)
line5, = ax4.plot([],[],color='blue',lw=2)
line6, = ax4.plot([],[],color='green',lw=2)
#ax1.set_aspect('equal', 'datalim')
ax4.set_xlim(0,1)
ax4.set_ylim(-500,700)
ax4.set_xticks(np.arange(0,1,0.2))
ax4.set_xticks([0,0.2,0.4,0.6,0.8,1])
ax4.axhline(y=0, color='r', linestyle='--')

def init():
    line1.set_data([],[])
    line2.set_data([],[])
    line21.set_data([],[])
    line3.set_data([],[])
    line4.set_data([],[])
    line5.set_data([],[])
    line6.set_data([],[])
    return line1,

def animate(i):
    x = var.b_h/var.b_l
    var.g_h10 =  var.g_l10*(i/100)
    var.g_l20 = var.g_h20*(i/100)
    P   = POWER(var)
    R   = np.array(R00(var))
    P_0 = np.array(P0(var))
    Q_d = R*P_0
    Q_l = Ql(var,P,Q_d)
    Q_h = Qh(var,P,Q_d)
    n   = EFF(P,Q_h)
    a =np.transpose(P)
    b = np.transpose(Q_h)
    c = np.transpose(Q_l)
    d = np.transpose(Q_d)
    e = np.transpose(n)
    y1 = n[39]   
    y2 = P[39]
    y21 = 1- np.sqrt(var.b_h/var.b_l)
    fig.suptitle(\
    r'$\frac{\gamma_h(\epsilon_{10})}{\gamma_h(\epsilon_{20})},\frac{\gamma_l(\epsilon_{20})}{\gamma_l(\epsilon_{10})}=%.4f$'%(i/100)\
                 +r'$ \;\; \frac{\Lambda}{\Lambda_{max}}= %.2f$'%(var.lw[39]/max(var.lw)),\
                fontsize= 13)
    
    line1.set_data(x,a[39])
    line1.set_label('P')
    line2.set_data(x,e[39])
    line2.set_label(r'$\eta$')
    line21.set_data(x,y21)
    line21.set_label(r'$\eta_{CA}$')
    line3.set_data(x,b[39])
    line3.set_label(r'$\dot Q_h$')
    line4.set_data(x,d[39])
    line4.set_label(r'$ \dot Q_d$')
    line5.set_data(x,c[39])
    line5.set_label(r'$\dot Q_l$')
    line6.set_data(x,d[39])
    line6.set_label(r'$\dotQ_d$')
    
      
    ax1.legend(bbox_to_anchor=(1, .8), loc='lower right', borderaxespad=0, fontsize=9)
    ax2.legend(bbox_to_anchor=(1, .8), loc='lower right', borderaxespad=0, fontsize=9)
    ax3.legend(bbox_to_anchor=(1, .7), loc='lower right', borderaxespad=0, fontsize=9)
    ax4.legend(bbox_to_anchor=(1, .7), loc='lower right', borderaxespad=0, fontsize=9)
     
    plt.tight_layout(pad= 1,w_pad =.02, h_pad =.200)
    return line1,


#anim.save(os.path.join(my_path,'lambda_opt.gif'), writer="imagemagick")
anim = FuncAnimation(fig, animate, frames= step, interval=8, blit=True)
#anim.save('../../../3level_img/beta_opt.gif', writer='imagemagick', fps=90)







In [ ]:
%matplotlib nbagg
import numpy as np
from matplotlib import pyplot as plt
from matplotlib.animation import FuncAnimation
plt.style.use('seaborn-pastel')


fig = plt.figure()
ax = plt.axes(xlim=(0, 4), ylim=(-2, 2))
line, = ax.plot([], [], lw=3)

def init():
    line.set_data([], [])
    return line,
def animate(i):
    x = np.linspace(0, 4, 1000)
    y = np.sin(2 * np.pi * (x - 0.01 * i))
    line.set_data(x, y)
    return line,

anim = FuncAnimation(fig, animate, init_func=init,
                               frames=200, interval=20, blit=True)

plt.show()